In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import confusion_matrix, classification_report

In [67]:
df = pd.read_csv('spam.csv',encoding='latin-1')

In [68]:
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Dikarenakan terdapat missing value dan juga hasil dari analisa saya bahwa sebenarnya kolom v2 dengan unnamed 2, 3, dan 4 saling memiliki hubungan atau sama maka saya akan lakukan join terhadap v2.
selain melakukan join v2 saya juga mengganti value dari v1 dari ham dan spam menjadi 0 dan 1 untuk ham adalah 0 dan untuk spam adalah 1 bertujuan untuk mempermudah proses

In [69]:
df['v2'] = df.apply(lambda row: ' '.join([str(row['v2']), str(row['Unnamed: 2']), str(row['Unnamed: 3']), str(row['Unnamed: 4'])]) if pd.notnull(row['Unnamed: 2']) else row['v2'], axis=1)
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
df = df[['v1', 'v2']]
df['v1'] = df['v1'].map({'ham': 0, 'spam': 1})

In [70]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Pra-pemrosesan teks
max_words = 10000
max_len = 500

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data['v2'])

X_train = tokenizer.texts_to_sequences(train_data['v2'])
X_test = tokenizer.texts_to_sequences(test_data['v2'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

y_train = train_data['v1']
y_test = test_data['v1']

In [71]:
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\Renz\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [73]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 33s 199ms/step - accuracy: 0.9091 - loss: 0.2734 - val_accuracy: 0.9830 - val_loss: 0.0734
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 26s 186ms/step - accuracy: 0.9916 - loss: 0.0262 - val_accuracy: 0.9803 - val_loss: 0.0598
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 26s 188ms/step - accuracy: 0.9980 - loss: 0.0081 - val_accuracy: 0.9821 - val_loss: 0.0611
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 26s 187ms/step - accuracy: 0.9997 - loss: 0.0016 - val_accuracy: 0.9830 - val_loss: 0.0754
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 26s 186ms/step - accuracy: 1.0000 - loss: 3.9570e-04 - val_accuracy: 0.9839 - val_loss: 0.0795


In [74]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')


35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.9917 - loss: 0.0430
Loss: 0.0795389711856842
Accuracy: 0.9838564991950989


In [75]:

# Prediksi
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()

# confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_mat)

#report
print("\nReport")
print(classification_report(y_test, y_pred))

# Menghitung akurasi
accuracy = (conf_mat[0, 0] + conf_mat[1, 1]) / np.sum(conf_mat)
print(f"\nAccuracy: {accuracy}")


35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 107ms/step
Confusion Matrix:
[[964   1]
 [ 17 133]]

Report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.99      0.89      0.94       150

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115


Accuracy: 0.9838565022421525


Pada pembentukan model dapat berubah seiring penggantian max_words pada max_words bertujuan untuk memilih jumlah kata umumnya digunakan 10000 sampai dengan 100000 namun hal tersebut bisa tergantung dari dataset tersebut. pada max_len juga dapat berpengaruh pada akurasi dari model namun umumnya max_len menggunakan 50 hingga 500. pada kasus ini digunakan max_words sebanyak 10000 dan pada max_len 500 selain mempengaruhi kinerja dari perangkat namun saya sudah melakukan beberapa kali percobaan dari 1000 max_words dan 50 max_len namun mendapatkan akurasi sekitar 0.8 oleh sebab itu saya mengganti dengan yang lebih besar. selain itu juga terdapat beberapa miss diantaranya terdapat 1 dari total false positiv yang berarti salah untuk memprediksi nilai spam dan terdapat 17 yang merupakan salah memprediksi nilai dari ham.